# Voice Classification

In this dataset we have several thousand voice recordings from people who auditioned for a European singing competition. Our job is to create an algorithm that will seperate male and female contestants for seperate judging. Variables like pitch, tempo etc are there which will help us determine what to do.


In [63]:
#Importing packages 
import pandas as pd
import numpy as np
import os
import math
from scipy import stats
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, auc, balanced_accuracy_score, confusion_matrix, f1_score, precision_score, average_precision_score, roc_auc_score,  recall_score,  precision_recall_curve 
import warnings
warnings.filterwarnings("ignore")

In [64]:
pip install statsmodels --upgrade

Note: you may need to restart the kernel to use updated packages.


In [18]:
#getting the dataset
voice=pd.read_csv("C:\\Users\\sujoydutta\\Desktop\\Data analysis\\Python\\Datasets_Py\\Classifier\\voice-classification.csv")
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [19]:
#shape of the dataset
voice.shape

(3168, 21)

In [20]:
#dataset info
voice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

In [21]:
#seeing null values
voice.isnull().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [26]:
#summary stats
voice.describe().round(3)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
count,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000,3168.000
mean,0.181,0.057,0.186,0.140,0.225,0.084,3.140,36.568,0.895,0.408,0.165,0.181,0.143,0.037,0.259,0.829,0.053,5.047,4.995,0.174
std,0.030,0.017,0.036,0.049,0.024,0.043,4.241,134.929,0.045,0.178,0.077,0.030,0.032,0.019,0.030,0.525,0.063,3.521,3.520,0.119
min,0.039,0.018,0.011,0.000,0.043,0.015,0.142,2.068,0.739,0.037,0.000,0.039,0.056,0.010,0.103,0.008,0.005,0.008,0.000,0.000
25%,0.164,0.042,0.170,0.111,0.209,0.043,1.650,5.670,0.862,0.258,0.118,0.164,0.117,0.018,0.254,0.420,0.008,2.070,2.045,0.100
50%,0.185,0.059,0.190,0.140,0.226,0.094,2.197,8.318,0.902,0.396,0.187,0.185,0.141,0.046,0.271,0.766,0.023,4.992,4.945,0.139
75%,0.199,0.067,0.211,0.176,0.244,0.114,2.932,13.649,0.929,0.534,0.221,0.199,0.170,0.048,0.277,1.177,0.070,7.008,6.992,0.209
max,0.251,0.115,0.261,0.247,0.273,0.252,34.725,1309.613,0.982,0.843,0.280,0.251,0.238,0.204,0.279,2.958,0.459,21.867,21.844,0.932


In [41]:
#initialazing encoder and scaler
gender_encoder = LabelEncoder()
scaler=StandardScaler()

In [28]:
#separating predictor variables
X= voice.iloc[:,:-1]
print(voice.shape)
print(X.shape)


(3168, 21)
(3168, 20)


In [43]:
#scaling the predictors
scaler.fit(X)
X= scaler.transform(X)

X

array([[-4.04924806,  0.4273553 , -4.22490077, ..., -1.43142165,
        -1.41913712, -1.45477229],
       [-3.84105325,  0.6116695 , -3.99929342, ..., -1.41810716,
        -1.4058184 , -1.01410294],
       [-3.46306647,  1.60384791, -4.09585052, ..., -1.42920257,
        -1.41691733, -1.06534356],
       ...,
       [-1.29877326,  2.32272355, -0.05197279, ..., -0.5992661 ,
        -0.58671739,  0.17588664],
       [-1.2452018 ,  2.012196  , -0.01772849, ..., -0.41286326,
        -0.40025537,  1.14916112],
       [-0.51474626,  2.14765111, -0.07087873, ..., -1.27608595,
        -1.2637521 ,  1.47567886]])

In [34]:
#creating target variable
y= voice.iloc[:,-1]
y

0         male
1         male
2         male
3         male
4         male
         ...  
3163    female
3164    female
3165    female
3166    female
3167    female
Name: label, Length: 3168, dtype: object

In [38]:
#transforming the target variable
y= gender_encoder.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 0])

In [51]:
#Splitting the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.25, random_state=100)


In [52]:
#creating the model 
svc_model=SVC()
svc_model.fit(X_train,y_train)

SVC()

In [53]:
#predictions 
y_pred=svc_model.predict(X_test)
y_pred

array([1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,

In [72]:
#Accuracy score
print("The Accuracy score of the model is", accuracy_score(y_test,y_pred).round(2)*100,"per cent!")

The Accuracy score of the model is 98.0 per cent!


In [73]:
#confusion matrix
print(confusion_matrix(y_test,y_pred))

[[387  11]
 [  7 387]]


In [78]:
#creating grid search parameters
param_grid= {'C':[0.1,1,10,100], 'gamma':[1,0.1,0.01,0.001]}

In [79]:
#making Grid search for further optimization
grid=GridSearchCV(SVC(),param_grid,refit=True, verbose=2)

In [80]:
#fitting the model
grid.fit(X_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END .....................................C=0.1, gamma=1; total time=   0.3s
[CV] END .....................................C=0.1, gamma=1; total time=   0.3s
[CV] END .....................................C=0.1, gamma=1; total time=   0.3s
[CV] END .....................................C=0.1, gamma=1; total time=   0.3s
[CV] END .....................................C=0.1, gamma=1; total time=   0.3s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ...................................C=0.1, gamma=0.1; total time=   0.1s
[CV] END ..................................C=0.1, gamma=0.01; total time=   0.1s
[CV] END ..................................C=0.1

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': [1, 0.1, 0.01, 0.001]},
             verbose=2)

In [81]:
#making predictions
grid_pred=grid.predict(X_test)
grid_pred

array([1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,

In [82]:
#Accuracy score
print("The Accuracy score of the model is", accuracy_score(y_test,grid_pred).round(2)*100,"per cent!")

The Accuracy score of the model is 97.0 per cent!


In [84]:
#classification report
print(classification_report(y_test,grid_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       398
           1       0.97      0.97      0.97       394

    accuracy                           0.97       792
   macro avg       0.97      0.97      0.97       792
weighted avg       0.97      0.97      0.97       792



In [85]:

#confusion matrix
print(confusion_matrix(y_test,grid_pred))

[[387  11]
 [ 10 384]]
